In [1]:
import numpy as np

In [2]:
def text_to_words(text, stop_word_pass='../stopwords/Japanese.txt'):
    # stopword listをつくる
    stopword_list = []
    with open(stop_word_pass, 'r') as f:
        stopword_list = f.readlines()
        
    stopword_list = [x.strip() for x in stopword_list if x.strip()] 
    #形態素解析を始める
    m = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    m.parse('')
    #text = normalize_text(text)
    text = mojimoji.zen_to_han(text, kana=False)
    m_text = m.parse(text)
    basic_words = []
    #mecabの出力結果を単語ごとにリスト化
    m_text = m_text.split('\n')
    for row in m_text:
        #Tab区切りで形態素、その品詞等の内容と分かれているので単語部のみ取得
        word = row.split("\t")[0]
        #最終行はEOS
        if word == 'EOS':
            break
        else:
            pos = row.split('\t')[1]
            slice_ = pos.split(',')
            #品詞を取得する
            parts = slice_[0]
            if parts == '記号':
                continue

            #活用語の場合は活用指定ない原型を取得する。
            elif slice_[0] in ('形容詞', '動詞') and slice_[-3] not in stopword_list:
                    basic_words.append(slice_[-3])

            #活用しない語についてはそのままの語を取得する
            elif slice_[0] =='名詞' and word not in stopword_list:
                basic_words.append(word)

    basic_words = ' '.join(basic_words)
    return basic_words

# 共起行列

In [3]:
def co_matrix(corpus, sep=' ', window_size=1):
    word_to_id = {}
    id_to_word = {}
    cnt = 0
    # Wordにidを振る
    for co in corpus:
        for c in co.split(sep):
            if c not in word_to_id:
                word_to_id[c] = cnt
                id_to_word[cnt] = c
                cnt += 1
    
    vocab_size = len(word_to_id)
    #出てきた単語数の正方行列を作成する。
    co_matrix = np.zeros((vocab_size, vocab_size))
    for co in corpus:
        corpus_words = co.split(sep)
        #文書のword数をセット
        corpus_size = len(corpus_words)
        for idx, c in enumerate(corpus_words):
            # wordのidを取得する
            center_index = word_to_id[c]
            # window_sizeの数だけループ
            for diff in range(1, window_size+1):
                # 対象の単語より前側を共起させる
                if idx - diff >=0:
                    left_word = corpus_words[idx - diff]
                    left_index = word_to_id[left_word]
                    co_matrix[center_index, left_index] +=1
                    
                # 対象の単語より後ろ側を共起させる
                if idx + diff < corpus_size:
                    right_word = corpus_words[idx + diff]
                    right_index = word_to_id[right_word]
                    co_matrix[center_index, right_index] += 1
                    
    return word_to_id, id_to_word, co_matrix
            

In [4]:
co_matrix(['私 東京駅 行く 東京駅 山手線 乗る'])

({'私': 0, '東京駅': 1, '行く': 2, '山手線': 3, '乗る': 4},
 {0: '私', 1: '東京駅', 2: '行く', 3: '山手線', 4: '乗る'},
 array([[0., 1., 0., 0., 0.],
        [1., 0., 2., 1., 0.],
        [0., 2., 0., 0., 0.],
        [0., 1., 0., 0., 1.],
        [0., 0., 0., 1., 0.]]))

# 相互情報量

$$
\textrm{PMI} \left( x, y \right) = \log_{2} \frac{P \left(x,y \right)}{P \left(x \right) P \left(y \right)} \\
\textrm{PPMI} \left( x, y \right) = \max \left[0, \textrm{PMI}\left( x, y \right) \right]
$$

In [5]:
def ppmi(C, delta=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    # 単語ごとの個数の合計を求める。
    S = np.sum(C, axis=0)
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / S[i]*S[j] + delta)
            M[i, j] = max(0, pmi)
            
    return M

In [6]:
_, _,m = co_matrix(['私 東京駅 行く', '東京駅 山手線 乗る','東京駅 品川駅 東海道新幹線 一駅','東京駅 大宮 大宮 東北新幹線 乗り換える'], window_size=1)
print(m)
ppmi(m)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 2. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


array([[0.       , 6.7813597, 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       ],
       [2.1375036, 0.       , 2.1375036, 3.1375036, 0.       , 3.1375036,
        0.       , 0.       , 4.1375036, 0.       , 0.       ],
       [0.       , 6.7813597, 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 5.7813597, 0.       , 0.       , 3.4594316, 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 5.4594316, 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 5.7813597, 0.       , 0.       , 0.       , 0.       ,
        4.4594316, 0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 4.4594316,
        0.       , 3.4594316, 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.  

# 主成分分析
主成分分析はScikit-learnに実装されている。

In [7]:
from sklearn.decomposition import PCA

# 文書の準備
今回はppmiの行列を次元削減することを考える。

In [8]:
corpus = ['東京駅 行く', \
          '東京駅 山手線 乗る',\
          '東京駅 品川駅 東海道新幹線 一駅',\
          '東京駅 大宮 大宮 東北新幹線 乗り換える']
_, id_to_word, m = co_matrix(corpus)
pp_m = ppmi(m)
print(pp_m)

[[0.       2.321928 3.321928 0.       3.321928 0.       0.       4.321928
  0.       0.      ]
 [6.321928 0.       0.       0.       0.       0.       0.       0.
  0.       0.      ]
 [5.321928 0.       0.       3.321928 0.       0.       0.       0.
  0.       0.      ]
 [0.       0.       5.321928 0.       0.       0.       0.       0.
  0.       0.      ]
 [5.321928 0.       0.       0.       0.       4.321928 0.       0.
  0.       0.      ]
 [0.       0.       0.       0.       4.321928 0.       3.321928 0.
  0.       0.      ]
 [0.       0.       0.       0.       0.       5.321928 0.       0.
  0.       0.      ]
 [4.321928 0.       0.       0.       0.       0.       0.       5.321928
  3.321928 0.      ]
 [0.       0.       0.       0.       0.       0.       0.       5.321928
  0.       3.321928]
 [0.       0.       0.       0.       0.       0.       0.       0.
  5.321928 0.      ]]


## 次元圧縮の実戦

In [9]:
pca = PCA(n_components=2)
co_text_pca = pca.fit_transform(pp_m)
co_text_pca

array([[ 4.4967055 ,  0.759573  ],
       [-4.124335  ,  0.8454847 ],
       [-3.801812  ,  0.5828286 ],
       [ 2.4012997 , -2.3495476 ],
       [-4.326973  , -1.303419  ],
       [ 2.215092  , -2.5340316 ],
       [-0.41045034, -3.6734424 ],
       [-0.5296067 ,  4.8526306 ],
       [ 3.0744364 ,  2.9625516 ],
       [ 1.005643  , -0.14262933]], dtype=float32)

## 寄与率を求めてみる

In [10]:
pca.explained_variance_ratio_

array([0.32518077, 0.21602066], dtype=float32)

# Word2Vec
今回の講座ではgensimのWord2Vecを用いる。  

モジュールがインストールされていない場合にはこちらを行う。  
\$ conda install -c conda-forge gensim

In [11]:
from gensim.models import word2vec
import MeCab
import mojimoji
import pandas as pd

/Users/tomone_hata/opt/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [12]:
df_input = pd.read_csv('../data/kokkai.csv', header=0)
corpus = [x.split(' ') for x in \
          df_input['text'].map(text_to_words).values.tolist()]

In [13]:
corpus[1]

['委員長',
 '礒崎哲史',
 '君',
 '経済産業委員会',
 '開会',
 'いたす',
 '委員',
 '異動',
 '報告',
 'いたす',
 '昨日',
 '三木亨',
 '君',
 '斎藤嘉隆',
 '君',
 '高橋はるみ',
 '君',
 '委員',
 '辞任',
 'する',
 'れる',
 '補欠',
 '蓮舫',
 '君',
 '山田修路',
 '君',
 '森屋宏',
 '君',
 '選任',
 'する',
 'れる']

## モデル作成
モデル作成を行い、このモデルを保管する。 　
- vector_size→次元数。
- min_count→何回以上出現した単語を対象にするか。
- window→前後どこまでを対象とするか。
- sg→1はski-gramそれ以外はCBOW

In [16]:
model = word2vec.Word2Vec(corpus, vector_size=200, min_count=3, window=5, sg=1)
model.save('../data/kokkai_model.model')

### ベクトルの出力

In [17]:
model.wv['アベノミクス']

array([ 0.715846  , -0.0381277 ,  0.10917306,  0.04000651, -0.01242358,
       -0.1698194 ,  0.07849717,  0.78439933, -0.13846494, -0.24546133,
        0.1452493 , -0.20151155,  0.39743558,  0.24011305, -0.2070299 ,
       -0.12100703, -0.31195468,  0.24189197, -0.37388372,  0.0634452 ,
        0.19018419, -0.18734032,  0.0891735 , -0.3613485 , -0.13861099,
        0.25699368,  0.233531  , -0.22662114,  0.14014769,  0.19815141,
        0.30167645,  0.25850433, -0.09452258,  0.19625859, -0.07373939,
        0.33976173,  0.4273567 , -0.0474168 , -0.32272398, -0.04161372,
        0.27649894, -0.00457198,  0.26991227, -0.2826261 ,  0.06118533,
        0.13170995,  0.07399867, -0.51248163,  0.09510687,  0.13255215,
       -0.58309484, -0.23750968,  0.02228343,  0.34231067,  0.43735185,
       -0.00103069, -0.18427412, -0.39359868, -0.05317821, -0.40725893,
        0.3794122 ,  0.10500915,  0.40858188, -0.14759625, -0.48479316,
       -0.25435013,  0.00243019,  0.3064391 , -0.31338617,  0.12

In [19]:
print(model.wv.similarity('アベノミクス', '三本の矢'))

0.7852021


In [20]:
similar_words = model.wv.most_similar(positive=['日本経済'], topn=10)
similar_words

[('内需', 0.8037256002426147),
 ('屋台骨', 0.7968873977661133),
 ('地方経済', 0.7805672883987427),
 ('足下', 0.7598000168800354),
 ('低迷', 0.7580502033233643),
 ('停滞', 0.7564107775688171),
 ('腰折れ', 0.7562618255615234),
 ('デフレマインド', 0.7505618333816528),
 ('不透明感', 0.7412660121917725),
 ('破滅', 0.7406598925590515)]

# TF-IDF

$$
\textrm{tf}_{i,j} = \frac{文書 d_{i}に含まれるt_{j}の個数}{文書d_{i}内のすべての単語の個数} = \frac{\left| t_{j} \in d_{i} \right| }{\sum_{t_{j} \in d_{j}}}
$$

$$
\textrm{idf}_{j} = \log \left( \frac{全文書数+1}{t_{j}が含まれる個数 +1} \right) + 1 = \log \left( \frac{\left| D \right| +1}{\left| \left\{ d : t_{j} \in d \right\} \right| +1 } \right) + 1
$$


で与えられtf-idfはtf値とidfの乗算で表され、それを文書が行、語彙が列として行列形式で並べて求められる。

$$
\left(
\begin{array}{cccc}
\textrm{tdidf}_{11} & \textrm{tdidf}_{12} & \cdots & \textrm{tdidf}_{1n} \\
\textrm{tdidf}_{21} & \textrm{tdidf}_{22} & \cdots & \textrm{tdidf}_{2n} \\
\cdots \\
\textrm{tdidf}_{m1} & \textrm{tdidf}_{n2} & \cdots & \textrm{tdidf}_{mn}
\end{array}
\right)
$$

今回使用するライブラリでは文書ごとにtf-idfが1に規格化されて出力される。$i$番目の文書のベクトルは

$$
\textbf{x}_{i} = \frac{1}{\sqrt{\sum_{k=1}^n \textrm{tdidf}_{ik}^{2}}} \left(
\begin{array}{cccc}
\textrm{tdidf}_{i1} & \textrm{tdidf}_{i2} & \cdots & \textrm{tdidf}_{n} 
\end{array}
\right)
$$
となる。CountVectorizerのライブラリではmax_df, min_dfというオプションがあり、max_df=0.8などと指定すると全文書の8割以上出現するものを無視することができる(min_dfも同様)。自然数で指定した場合は割合ではなく回数となる。

In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd
corpus = ['東京駅 行く', \
          '東京駅 山手線 乗る',\
          '東京駅 品川駅 東海道新幹線 一駅',\
          '東京駅 大宮 大宮 東北新幹線 乗り換える']
vec = CountVectorizer()
X = vec.fit_transform(corpus)
print('Bag of Words')
display(pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index=corpus))
print('tfidf')
tfidf = TfidfTransformer()
X_tfidf = tfidf.fit_transform(X)
display(pd.DataFrame(X_tfidf.toarray(), columns=vec.get_feature_names(), index=corpus))

print('max_df=0.5')
vec = CountVectorizer(max_df=0.5)
X = vec.fit_transform(corpus)
print('Bag of Words')
display(pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index=corpus))
print('tfidf')
tfidf = TfidfTransformer()
X_tfidf = tfidf.fit_transform(X)
display(pd.DataFrame(X_tfidf.toarray(), columns=vec.get_feature_names(), index=corpus))

print('min_df=0.3')
vec = CountVectorizer(min_df=0.3)
X = vec.fit_transform(corpus)
print('Bag of Words')
display(pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index=corpus))
print('tfidf')
tfidf = TfidfTransformer()
X_tfidf = tfidf.fit_transform(X)
display(pd.DataFrame(X_tfidf.toarray(), columns=vec.get_feature_names(), index=corpus))

Bag of Words


,一駅,乗り換える,乗る,品川駅,大宮,山手線,東京駅,東北新幹線,東海道新幹線,行く
東京駅 行く,0,0,0,0,0,0,1,0,0,1
東京駅 山手線 乗る,0,0,1,0,0,1,1,0,0,0
東京駅 品川駅 東海道新幹線 一駅,1,0,0,1,0,0,1,0,1,0
東京駅 大宮 大宮 東北新幹線 乗り換える,0,1,0,0,2,0,1,1,0,0


tfidf


,一駅,乗り換える,乗る,品川駅,大宮,山手線,東京駅,東北新幹線,東海道新幹線,行く
東京駅 行く,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.462637,0.000000,0.000000,0.886548
東京駅 山手線 乗る,0.000000,0.000000,0.663385,0.000000,0.000000,0.663385,0.346182,0.000000,0.000000,0.000000
東京駅 品川駅 東海道新幹線 一駅,0.552805,0.000000,0.000000,0.552805,0.000000,0.000000,0.288477,0.000000,0.552805,0.000000
東京駅 大宮 大宮 東北新幹線 乗り換える,0.000000,0.399288,0.000000,0.000000,0.798575,0.000000,0.208365,0.399288,0.000000,0.000000


max_df=0.5
Bag of Words


...

,一駅,乗り換える,乗る,品川駅,大宮,山手線,東北新幹線,東海道新幹線,行く
東京駅 行く,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,1.0
東京駅 山手線 乗る,0.00000,0.000000,0.707107,0.00000,0.000000,0.707107,0.000000,0.00000,0.0
東京駅 品川駅 東海道新幹線 一駅,0.57735,0.000000,0.000000,0.57735,0.000000,0.000000,0.000000,0.57735,0.0
東京駅 大宮 大宮 東北新幹線 乗り換える,0.00000,0.408248,0.000000,0.00000,0.816497,0.000000,0.408248,0.00000,0.0


min_df=0.3
Bag of Words


,東京駅
東京駅 行く,1
東京駅 山手線 乗る,1
東京駅 品川駅 東海道新幹線 一駅,1
東京駅 大宮 大宮 東北新幹線 乗り換える,1


tfidf


,東京駅
東京駅 行く,1.0
東京駅 山手線 乗る,1.0
東京駅 品川駅 東海道新幹線 一駅,1.0
東京駅 大宮 大宮 東北新幹線 乗り換える,1.0


# Doc2Vec
gensimのDoc2Vecを用いる。

In [22]:
import pandas as pd
import MeCab
import mojimoji
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

df_input = pd.read_csv('../data/kokkai.csv', header=0)
df_input['text_ana'] = df_input['text'].map(text_to_words)

In [23]:
df_input.head()

,date,house,meeting,speech_order,text,text_ana
0,2019-12-03,参議院,経済産業委員会,0,令和元年十二月三日（火曜日）\r\n 午前十時開会\r\n ───────────...,令和元年 十二月三日 火曜日 午前 十時 開会 委員 異動 十一月二十八日 辞任 補欠 選任...
1,2019-12-03,参議院,経済産業委員会,1,○委員長（礒崎哲史君） ただいまから経済産業委員会を開会いたします。\r\n 委員の異動につ...,委員長 礒崎哲史 君 経済産業委員会 開会 いたす 委員 異動 報告 いたす 昨日 三木亨 ...
2,2019-12-03,参議院,経済産業委員会,2,○委員長（礒崎哲史君） 政府参考人の出席要求に関する件についてお諮りいたします。\r\n 外...,委員長 礒崎哲史 君 政府参考人 出席 要求 件 諮る いたす 外国為替及び外国貿易法 第十...
3,2019-12-03,参議院,経済産業委員会,3,○委員長（礒崎哲史君） 御異議ないと認め、さよう決定いたします。\r\n ──────...,委員長 礒崎哲史 君 異議 ない 認める 決定 いたす
4,2019-12-03,参議院,経済産業委員会,4,○委員長（礒崎哲史君） 外国為替及び外国貿易法第十条第二項の規定に基づき、北朝鮮を仕向地とす...,委員長 礒崎哲史 君 外国為替及び外国貿易法 第十条 項 規定 基づく 北朝鮮 仕向 する ...


## 前処理
Doc2Vecのための準備を行う。TaggedDocumentを用いてデータを準備する。  
TaggedDocumentは以下の形式で作成する。  
&nbsp;&nbsp;TaggedDocument(分かち書きした文書(単語ごとのリスト),タグ(文書ID))

ここではタグは日付+参議院or衆議院+会議名とする。

In [24]:
keys = df_input[['date', 'house', 'meeting']].drop_duplicates().values.tolist()
tagged_corpus = []
for key_list in keys:
    df = df_input[(df_input['date'] == key_list[0]) & \
                  (df_input['house'] == key_list[1]) & \
                  (df_input['meeting'] == key_list[2]) \
                 ]
    df = df.sort_values(by=['speech_order'])
    sentence = ' \n '.join(df['text_ana'].values.tolist())
    sentence = sentence.split(' ')
    tag = ['{}{}{}'.format(key_list[0], key_list[1], key_list[2])]
    tagged_corpus.append(TaggedDocument(sentence, tag))
                         

## モデル作成
モデル作成を行い、このモデルを保管する。 　
- vecrsize→次元数。
- window→前後どこまでを対象とするか。
- dm→1はdmpvそれ以外はDBOW

In [27]:
model = Doc2Vec(documents=tagged_corpus, vector_size=300, window=3, dm = 1)
#'2017-05-29_参議院_本会議'
model.docvecs.most_similar('2019-12-03参議院経済産業委員会')

/Users/tomone_hata/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  This is separate from the ipykernel package so we can avoid doing imports until


[('2019-11-22衆議院経済産業委員会', 0.6214159727096558),
 ('2019-11-14参議院経済産業委員会', 0.5819849967956543),
 ('2019-05-16参議院経済産業委員会', 0.5752360224723816),
 ('2019-03-14参議院経済産業委員会', 0.5379735827445984),
 ('2019-05-28参議院経済産業委員会', 0.5098767280578613),
 ('2019-05-23参議院経済産業委員会', 0.5002453923225403),
 ('2019-12-09参議院経済産業委員会', 0.4995992183685303),
 ('2019-06-26参議院経済産業委員会', 0.49713918566703796),
 ('2019-08-05参議院経済産業委員会', 0.48783719539642334),
 ('2019-05-21参議院経済産業委員会', 0.4795587360858917)]

In [23]:
model.docvecs['2019-12-03参議院経済産業委員会'].shape

(300,)